###  데이터 불러오기

In [1]:
import pickle
import gzip
import pandas as pd

In [2]:
# col 생략 없이 출력
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

C:\Users\elyse\AppData\Local\Temp\ipykernel_13540\94279240.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
with gzip.open('땡큐스티커.pickle','rb') as f:
    data = pickle.load(f)
df_sticker = pd.DataFrame(data)

with gzip.open('접속횟수.pickle','rb') as f:
    data = pickle.load(f)
df_access = pd.DataFrame(data)

with gzip.open('채팅팬.pickle','rb') as f:
    data = pickle.load(f)
df_fan = pd.DataFrame(data)

with gzip.open('채팅횟수.pickle','rb') as f:
    data = pickle.load(f)
df_chat= pd.DataFrame(data)

with gzip.open('celeb_list.pickle','rb') as f:
    data = pickle.load(f)
df_list = pd.DataFrame(data)



### df_chat에서 채팅최초일자, 채팅최근일자, 이탈여부 채팅기간(day/week), 누적채팅량, 최근 일자기준 1주일간 채팅량 구하기

In [4]:
df_chat

,chat_room_id,user_nickname,message_length,time
0,sendbird_group_channel_90584763_e7b41d72ca4a6efdef2d11c6c7efd4a9e9ed57a8,정다별이,1.0,2022-09-04 00:00:03+00:00
1,sendbird_group_channel_90584763_e7b41d72ca4a6efdef2d11c6c7efd4a9e9ed57a8,정다별이,9.0,2022-09-04 00:00:08+00:00
2,sendbird_group_channel_90584763_e7b41d72ca4a6efdef2d11c6c7efd4a9e9ed57a8,정다별이,10.0,2022-09-04 00:00:13+00:00
3,sendbird_group_channel_90584763_e7b41d72ca4a6efdef2d11c6c7efd4a9e9ed57a8,정다별이,12.0,2022-09-04 00:00:17+00:00
4,sendbird_group_channel_90584763_e7b41d72ca4a6efdef2d11c6c7efd4a9e9ed57a8,정다별이,5.0,2022-09-04 00:00:18+00:00
...,...,...,...,...
506037,sendbird_group_channel_94516740_608949752e067685ec73cfa3491f2b90ab79c542,김향단,9.0,2023-01-20 14:56:08+00:00
506038,sendbird_group_channel_92280528_bb1148216c54f7b138d6798262da032358a4f845,도라선장,12.0,2023-01-20 14:56:11+00:00
506039,sendbird_group_channel_100972710_3b096addf9b98292d4064af4c8c4b4dca828ef98,서2,10.0,2023-01-20 14:56:12+00:00
506040,sendbird_group_channel_92280528_bb1148216c54f7b138d6798262da032358a4f845,도라선장,10.0,2023-01-20 14:56:35+00:00


In [5]:
#df_tmp1 셀럽 가장 최근 채팅일자
df_tmp1 = df_chat.groupby('user_nickname',as_index=False).max()[['user_nickname','time']]
#날짜만 가져오기
df_tmp1['time'] = df_tmp1['time'].apply(lambda x :x.date)
#컬럼이름을 last_chat_date로 변경
df_tmp1.rename(columns = {'time':'last_celeb_chat_date'}, inplace=True)
df_tmp1

,user_nickname,last_celeb_chat_date
0,(ㅅ´ ˘ `)♡,2022-11-17
1,102,2023-01-18
2,104,2023-01-17
3,105,2023-01-17
4,106,2023-01-19
...,...,...
675,🍼한우유,2023-01-20
676,👿르릉이,2023-01-20
677,📢오늘의팬심 ver.03,2022-11-07
678,🔮뉸뉴,2023-01-18


In [6]:
#df_tmp2 셀럽 최초 채팅일자(기간중)
df_tmp2 = df_chat.groupby('user_nickname',as_index=False).min()[['user_nickname','time']]
#날짜만 가져오기
df_tmp2['time'] = df_tmp2['time'].apply(lambda x :x.date)
#컬럼이름을 last_chat_date로 변경
df_tmp2.rename(columns = {'time':'first_celeb_chat_date'}, inplace=True)
df_tmp2

,user_nickname,first_celeb_chat_date
0,(ㅅ´ ˘ `)♡,2022-11-17
1,102,2023-01-17
2,104,2023-01-17
3,105,2023-01-17
4,106,2023-01-17
...,...,...
675,🍼한우유,2022-12-21
676,👿르릉이,2022-11-27
677,📢오늘의팬심 ver.03,2022-09-04
678,🔮뉸뉴,2023-01-13


In [7]:
#df_tmp3 셀럽 누적 채팅수
df_tmp3 = df_chat.groupby('user_nickname',as_index=False).count()[['user_nickname','time']]
#컬럼이름을 chat_count로 변경
df_tmp3.rename(columns = {'time':'celeb_chat_count'}, inplace=True)
df_tmp3

,user_nickname,celeb_chat_count
0,(ㅅ´ ˘ `)♡,7
1,102,4
2,104,1
3,105,13
4,106,9
...,...,...
675,🍼한우유,261
676,👿르릉이,167
677,📢오늘의팬심 ver.03,2012
678,🔮뉸뉴,83


In [8]:
#임시 데이터 합치고, 채팅 최초~마지막 날짜 기간 구하기
df_tmp1 = df_tmp1.merge(df_tmp2,on='user_nickname')
df_tmp1['celeb_chat_period']=df_tmp1['last_celeb_chat_date']-df_tmp1['first_celeb_chat_date']
df_tmp1['celeb_chat_period'] = df_tmp1['celeb_chat_period'].dt.days+1
df_tmp1

,user_nickname,last_celeb_chat_date,first_celeb_chat_date,celeb_chat_period
0,(ㅅ´ ˘ `)♡,2022-11-17,2022-11-17,1
1,102,2023-01-18,2023-01-17,2
2,104,2023-01-17,2023-01-17,1
3,105,2023-01-17,2023-01-17,1
4,106,2023-01-19,2023-01-17,3
...,...,...,...,...
675,🍼한우유,2023-01-20,2022-12-21,31
676,👿르릉이,2023-01-20,2022-11-27,55
677,📢오늘의팬심 ver.03,2022-11-07,2022-09-04,65
678,🔮뉸뉴,2023-01-18,2023-01-13,6


In [9]:
#날짜로 몇주인지 구하는 함수
def div_week(num):
    if num <6:
        return 1
    elif num % 7 == 0:
        return int(num / 7)
    else:
        return int(num // 7+1)

In [10]:
df_tmp1['celeb_chat_week'] = df_tmp1['celeb_chat_period'].apply(div_week)
df_tmp1

,user_nickname,last_celeb_chat_date,first_celeb_chat_date,celeb_chat_period,celeb_chat_week
0,(ㅅ´ ˘ `)♡,2022-11-17,2022-11-17,1,1
1,102,2023-01-18,2023-01-17,2,1
2,104,2023-01-17,2023-01-17,1,1
3,105,2023-01-17,2023-01-17,1,1
4,106,2023-01-19,2023-01-17,3,1
...,...,...,...,...,...
675,🍼한우유,2023-01-20,2022-12-21,31,5
676,👿르릉이,2023-01-20,2022-11-27,55,8
677,📢오늘의팬심 ver.03,2022-11-07,2022-09-04,65,10
678,🔮뉸뉴,2023-01-18,2023-01-13,6,1


In [11]:
#df_tmp3도 합침
df_tmp1 = df_tmp1.merge(df_tmp3,on='user_nickname')
df_tmp1

,user_nickname,last_celeb_chat_date,first_celeb_chat_date,celeb_chat_period,celeb_chat_week,celeb_chat_count
0,(ㅅ´ ˘ `)♡,2022-11-17,2022-11-17,1,1,7
1,102,2023-01-18,2023-01-17,2,1,4
2,104,2023-01-17,2023-01-17,1,1,1
3,105,2023-01-17,2023-01-17,1,1,13
4,106,2023-01-19,2023-01-17,3,1,9
...,...,...,...,...,...,...
675,🍼한우유,2023-01-20,2022-12-21,31,5,261
676,👿르릉이,2023-01-20,2022-11-27,55,8,167
677,📢오늘의팬심 ver.03,2022-11-07,2022-09-04,65,10,2012
678,🔮뉸뉴,2023-01-18,2023-01-13,6,1,83


In [12]:
#주간 채팅 횟수 구하기
df_tmp1['avg_celeb_chat'] = round ( df_tmp1['celeb_chat_count'] /  df_tmp1['celeb_chat_week'] , 2)
df_tmp1

,user_nickname,last_celeb_chat_date,first_celeb_chat_date,celeb_chat_period,celeb_chat_week,celeb_chat_count,avg_celeb_chat
0,(ㅅ´ ˘ `)♡,2022-11-17,2022-11-17,1,1,7,7.00
1,102,2023-01-18,2023-01-17,2,1,4,4.00
2,104,2023-01-17,2023-01-17,1,1,1,1.00
3,105,2023-01-17,2023-01-17,1,1,13,13.00
4,106,2023-01-19,2023-01-17,3,1,9,9.00
...,...,...,...,...,...,...,...
675,🍼한우유,2023-01-20,2022-12-21,31,5,261,52.20
676,👿르릉이,2023-01-20,2022-11-27,55,8,167,20.88
677,📢오늘의팬심 ver.03,2022-11-07,2022-09-04,65,10,2012,201.20
678,🔮뉸뉴,2023-01-18,2023-01-13,6,1,83,83.00


In [13]:
from datetime import timedelta
import datetime

In [14]:
df_chat_date = df_chat.copy()
df_chat_date['time'] = df_chat['time'].apply(lambda x :x.date)

In [15]:
#최근 일주일간 채팅 구하기
tmp_list = []
for idx, row in df_tmp1.iterrows():
    day = row["last_celeb_chat_date"]+timedelta(days=-7)
    celeb = row["user_nickname"]
    tmp_list.append( df_chat_date[ (df_chat_date['user_nickname'] ==celeb) & (df_chat_date['time'] > day)].shape[0] )


In [16]:
df_tmp1['last_week_celeb_chat_count'] = tmp_list
df_tmp1

,user_nickname,last_celeb_chat_date,first_celeb_chat_date,celeb_chat_period,celeb_chat_week,celeb_chat_count,avg_celeb_chat,last_week_celeb_chat_count
0,(ㅅ´ ˘ `)♡,2022-11-17,2022-11-17,1,1,7,7.00,7
1,102,2023-01-18,2023-01-17,2,1,4,4.00,4
2,104,2023-01-17,2023-01-17,1,1,1,1.00,1
3,105,2023-01-17,2023-01-17,1,1,13,13.00,13
4,106,2023-01-19,2023-01-17,3,1,9,9.00,9
...,...,...,...,...,...,...,...,...
675,🍼한우유,2023-01-20,2022-12-21,31,5,261,52.20,100
676,👿르릉이,2023-01-20,2022-11-27,55,8,167,20.88,2
677,📢오늘의팬심 ver.03,2022-11-07,2022-09-04,65,10,2012,201.20,6
678,🔮뉸뉴,2023-01-18,2023-01-13,6,1,83,83.00,83


In [17]:
#df_tmp4 채팅방 셀럽 채팅 1건당 평균 길이
df_tmp4 = df_chat.groupby('user_nickname',as_index=False).sum()[['user_nickname','message_length']]
#컬럼이름 변경
df_tmp4.rename(columns = {'message_length':'celeb_chat_length'}, inplace=True)
df_tmp4['celeb_chat_avg_len'] = df_tmp4['celeb_chat_length']/   df_tmp1['celeb_chat_count']
df_tmp4

,user_nickname,celeb_chat_length,celeb_chat_avg_len
0,(ㅅ´ ˘ `)♡,57.0,8.142857
1,102,26.0,6.500000
2,104,2.0,2.000000
3,105,144.0,11.076923
4,106,48.0,5.333333
...,...,...,...
675,🍼한우유,5822.0,22.306513
676,👿르릉이,1573.0,9.419162
677,📢오늘의팬심 ver.03,178505.0,88.720179
678,🔮뉸뉴,615.0,7.409639


In [18]:
#df_tmp4 병합
df_tmp1 = df_tmp1.merge(df_tmp4,on='user_nickname')
df_tmp1

,user_nickname,last_celeb_chat_date,first_celeb_chat_date,celeb_chat_period,celeb_chat_week,celeb_chat_count,avg_celeb_chat,last_week_celeb_chat_count,celeb_chat_length,celeb_chat_avg_len
0,(ㅅ´ ˘ `)♡,2022-11-17,2022-11-17,1,1,7,7.00,7,57.0,8.142857
1,102,2023-01-18,2023-01-17,2,1,4,4.00,4,26.0,6.500000
2,104,2023-01-17,2023-01-17,1,1,1,1.00,1,2.0,2.000000
3,105,2023-01-17,2023-01-17,1,1,13,13.00,13,144.0,11.076923
4,106,2023-01-19,2023-01-17,3,1,9,9.00,9,48.0,5.333333
...,...,...,...,...,...,...,...,...,...,...
675,🍼한우유,2023-01-20,2022-12-21,31,5,261,52.20,100,5822.0,22.306513
676,👿르릉이,2023-01-20,2022-11-27,55,8,167,20.88,2,1573.0,9.419162
677,📢오늘의팬심 ver.03,2022-11-07,2022-09-04,65,10,2012,201.20,6,178505.0,88.720179
678,🔮뉸뉴,2023-01-18,2023-01-13,6,1,83,83.00,83,615.0,7.409639


In [19]:
#이탈여부 컬럼 추가 (마지막 채팅으로부터 일주일 이상). 
#마지막 20일은 24시까지가 아닌 15시까지의 데이터이므로 13일도 포함시킴

df_tmp1['churn'] = (df_tmp1['last_celeb_chat_date'] < datetime.date(2023,1,13) ).astype(int)
df_tmp1

,user_nickname,last_celeb_chat_date,first_celeb_chat_date,celeb_chat_period,celeb_chat_week,celeb_chat_count,avg_celeb_chat,last_week_celeb_chat_count,celeb_chat_length,celeb_chat_avg_len,churn
0,(ㅅ´ ˘ `)♡,2022-11-17,2022-11-17,1,1,7,7.00,7,57.0,8.142857,1
1,102,2023-01-18,2023-01-17,2,1,4,4.00,4,26.0,6.500000,0
2,104,2023-01-17,2023-01-17,1,1,1,1.00,1,2.0,2.000000,0
3,105,2023-01-17,2023-01-17,1,1,13,13.00,13,144.0,11.076923,0
4,106,2023-01-19,2023-01-17,3,1,9,9.00,9,48.0,5.333333,0
...,...,...,...,...,...,...,...,...,...,...,...
675,🍼한우유,2023-01-20,2022-12-21,31,5,261,52.20,100,5822.0,22.306513,0
676,👿르릉이,2023-01-20,2022-11-27,55,8,167,20.88,2,1573.0,9.419162,0
677,📢오늘의팬심 ver.03,2022-11-07,2022-09-04,65,10,2012,201.20,6,178505.0,88.720179,1
678,🔮뉸뉴,2023-01-18,2023-01-13,6,1,83,83.00,83,615.0,7.409639,0


### df_access에서 가입일 컬럼 추가

In [20]:
df_tmp5 = df_access.query('mp_event_name=="joinus succeeded"')
df_tmp5 =df_tmp5.groupby('user_nickname',as_index=False).first()[['user_nickname','time' ]]
df_tmp5['time'] = df_tmp5['time'].apply(lambda x :x.date)
#컬럼이름을 joinus_date로 변경
df_tmp5.rename(columns = {'time':'celeb_joinus_date'}, inplace=True)
df_tmp5

,user_nickname,celeb_joinus_date
0,.......,2022-11-23
1,102,2023-01-17
2,103,2023-01-17
3,104,2023-01-17
4,105,2023-01-17
...,...,...
509,흰나비,2023-01-02
510,히요코,2022-09-28
511,히응,2022-11-06
512,히차,2022-11-18


In [21]:
#df_tmp5 병합
df_tmp1 = df_tmp1.merge(df_tmp5, how='left', on='user_nickname')
df_tmp1

,user_nickname,last_celeb_chat_date,first_celeb_chat_date,celeb_chat_period,celeb_chat_week,celeb_chat_count,avg_celeb_chat,last_week_celeb_chat_count,celeb_chat_length,celeb_chat_avg_len,churn,celeb_joinus_date
0,(ㅅ´ ˘ `)♡,2022-11-17,2022-11-17,1,1,7,7.00,7,57.0,8.142857,1,NaN
1,102,2023-01-18,2023-01-17,2,1,4,4.00,4,26.0,6.500000,0,2023-01-17
2,104,2023-01-17,2023-01-17,1,1,1,1.00,1,2.0,2.000000,0,2023-01-17
3,105,2023-01-17,2023-01-17,1,1,13,13.00,13,144.0,11.076923,0,2023-01-17
4,106,2023-01-19,2023-01-17,3,1,9,9.00,9,48.0,5.333333,0,2023-01-17
...,...,...,...,...,...,...,...,...,...,...,...,...
675,🍼한우유,2023-01-20,2022-12-21,31,5,261,52.20,100,5822.0,22.306513,0,NaN
676,👿르릉이,2023-01-20,2022-11-27,55,8,167,20.88,2,1573.0,9.419162,0,NaN
677,📢오늘의팬심 ver.03,2022-11-07,2022-09-04,65,10,2012,201.20,6,178505.0,88.720179,1,NaN
678,🔮뉸뉴,2023-01-18,2023-01-13,6,1,83,83.00,83,615.0,7.409639,0,NaN


In [22]:
df_tmp1['celeb_joinus_date'] = df_tmp1['celeb_joinus_date'] .fillna('before 2022-09-04')

In [23]:
df_tmp1

,user_nickname,last_celeb_chat_date,first_celeb_chat_date,celeb_chat_period,celeb_chat_week,celeb_chat_count,avg_celeb_chat,last_week_celeb_chat_count,celeb_chat_length,celeb_chat_avg_len,churn,celeb_joinus_date
0,(ㅅ´ ˘ `)♡,2022-11-17,2022-11-17,1,1,7,7.00,7,57.0,8.142857,1,before 2022-09-04
1,102,2023-01-18,2023-01-17,2,1,4,4.00,4,26.0,6.500000,0,2023-01-17
2,104,2023-01-17,2023-01-17,1,1,1,1.00,1,2.0,2.000000,0,2023-01-17
3,105,2023-01-17,2023-01-17,1,1,13,13.00,13,144.0,11.076923,0,2023-01-17
4,106,2023-01-19,2023-01-17,3,1,9,9.00,9,48.0,5.333333,0,2023-01-17
...,...,...,...,...,...,...,...,...,...,...,...,...
675,🍼한우유,2023-01-20,2022-12-21,31,5,261,52.20,100,5822.0,22.306513,0,before 2022-09-04
676,👿르릉이,2023-01-20,2022-11-27,55,8,167,20.88,2,1573.0,9.419162,0,before 2022-09-04
677,📢오늘의팬심 ver.03,2022-11-07,2022-09-04,65,10,2012,201.20,6,178505.0,88.720179,1,before 2022-09-04
678,🔮뉸뉴,2023-01-18,2023-01-13,6,1,83,83.00,83,615.0,7.409639,0,before 2022-09-04


### 이탈/신규/중기/장기 고객 분류

In [24]:
tmp_list=[]
for idx,row in df_tmp1.iterrows():
    if row['churn'] == 1:
        tmp_list.append('churn_customer')
    elif row['celeb_chat_week'] < 6 :
        tmp_list.append('new_customer')
    elif row['celeb_chat_week'] > 15 :
        tmp_list.append('long_term')
    else:
        tmp_list.append('mid_term')

df_tmp1['celeb_type'] = tmp_list
df_tmp1

,user_nickname,last_celeb_chat_date,first_celeb_chat_date,celeb_chat_period,celeb_chat_week,celeb_chat_count,avg_celeb_chat,last_week_celeb_chat_count,celeb_chat_length,celeb_chat_avg_len,churn,celeb_joinus_date,celeb_type
0,(ㅅ´ ˘ `)♡,2022-11-17,2022-11-17,1,1,7,7.00,7,57.0,8.142857,1,before 2022-09-04,churn_customer
1,102,2023-01-18,2023-01-17,2,1,4,4.00,4,26.0,6.500000,0,2023-01-17,new_customer
2,104,2023-01-17,2023-01-17,1,1,1,1.00,1,2.0,2.000000,0,2023-01-17,new_customer
3,105,2023-01-17,2023-01-17,1,1,13,13.00,13,144.0,11.076923,0,2023-01-17,new_customer
4,106,2023-01-19,2023-01-17,3,1,9,9.00,9,48.0,5.333333,0,2023-01-17,new_customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,🍼한우유,2023-01-20,2022-12-21,31,5,261,52.20,100,5822.0,22.306513,0,before 2022-09-04,new_customer
676,👿르릉이,2023-01-20,2022-11-27,55,8,167,20.88,2,1573.0,9.419162,0,before 2022-09-04,mid_term
677,📢오늘의팬심 ver.03,2022-11-07,2022-09-04,65,10,2012,201.20,6,178505.0,88.720179,1,before 2022-09-04,churn_customer
678,🔮뉸뉴,2023-01-18,2023-01-13,6,1,83,83.00,83,615.0,7.409639,0,before 2022-09-04,new_customer


In [25]:
#결측치 확인
df_tmp1.isna().sum()

user_nickname                 0
last_celeb_chat_date          0
first_celeb_chat_date         0
celeb_chat_period             0
celeb_chat_week               0
celeb_chat_count              0
avg_celeb_chat                0
last_week_celeb_chat_count    0
celeb_chat_length             0
celeb_chat_avg_len            0
churn                         0
celeb_joinus_date             0
celeb_type                    0
dtype: int64

In [26]:
#csv 내보내기
df_tmp1.to_csv('cp2_셀럽_데이터.csv', index=False)